In [1]:
import os, shutil

persist_dir = "./chromadb_v3"
abs_dir = os.path.abspath(persist_dir)
print("Working dir:", os.getcwd())
print("Will remove:", abs_dir)
print("Exists?", os.path.exists(abs_dir))

if os.path.exists(abs_dir):
    # Xóa thư mục cũ (bỏ qua lỗi nếu có)
    shutil.rmtree(abs_dir, ignore_errors=True)
    print("Removed:", not os.path.exists(abs_dir))
else:
    print("No directory to remove.")


Working dir: d:\Project_self
Will remove: d:\Project_self\chromadb_v3
Exists? True
Removed: True


In [2]:
# check đường dẫn đúng để xóa db hỗ trợ code trên, để mỗi lần đọc 1 file pdf mới thì nó sẽ reset
import os
persist_dir = "./chromadb_v3"
print("Working dir:", os.getcwd())
print("Will remove:", os.path.abspath(persist_dir))
print("Exists?", os.path.exists(persist_dir))

Working dir: d:\Project_self
Will remove: d:\Project_self\chromadb_v3
Exists? False


In [3]:
import os
import shutil
import gc
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

def normalize_text(s: str) -> str:
    # ép thành bytes rồi decode utf-8, bỏ ký tự lạ
    return s.encode("utf-8", errors="ignore").decode("utf-8")

def build_chroma_from_single_pdf(
    pdf_path: str,
    persist_dir: str = "./chromadb_v3",
    chunk_size: int = 1500,
    chunk_overlap: int = 200,
    hf_model: str = "sentence-transformers/all-MiniLM-L6-v2",
    device: str | None = None,
    batch_size: int = 64
) -> Chroma:
    # 1) Xóa cache cũ
    if os.path.isdir(persist_dir):
        shutil.rmtree(persist_dir, ignore_errors=True)
    gc.collect()

    # 2) Load PDF
    assert os.path.isfile(pdf_path), f"File không tồn tại: {pdf_path}"
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    # 3) Normalize text trên từng document
    for d in docs:
        d.page_content = normalize_text(d.page_content)

    # 4) Split thành chunks
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = splitter.split_documents(docs)

    # 5) Embedding
    model_kwargs = {"device": device} if device else {}
    hf_emb = HuggingFaceEmbeddings(
        model_name=hf_model,
        model_kwargs=model_kwargs,
        encode_kwargs={"batch_size": batch_size}
    )

    # 6) Build Chroma store
    vectordb = Chroma.from_documents(
        documents=chunks,
        embedding=hf_emb,
        persist_directory=persist_dir
    )

    print(f"✅ Vector store đã sẵn sàng tại {persist_dir}")
    return vectordb

if __name__ == "__main__":
    pdf_file = r"D:\Project_self\pdf_place\CleanCode.pdf"
    vectordb = build_chroma_from_single_pdf(
        pdf_path    = pdf_file,
        persist_dir = "./chromadb_v3",
        device      = None   # hoặc "cuda"
    )

    # Thử truy vấn
    for doc in vectordb.similarity_search("Your query here", k=3):
        print(doc.page_content[:200].replace("\n"," "), "…")


✅ Vector store đã sẵn sàng tại ./chromadb_v3
T8 ....................................................................................16-275, 17-335 T9 ............................................................................................... …
G28 ..................................................................................15-262, 17-317 G29 ..................................................................................15-262, 17-31 …
331             case MARCH:   332             case APRIL:   333             case MAY:   334             case JUNE:   335             case JULY:   336             case AUGUST:   337             case SE …


In [4]:
from langchain.chains import RetrievalQA
from langchain.llms import GPT4All

\C:\Users\klein\AppData\Local\nomic.ai\GPT4All\Llama-3.2-1B-Instruct-Q4_0.gguf

In [5]:
# from gpt4all import GPT4All

# # 1. Khởi tạo model với đúng keyword args
# gpt = GPT4All(
#     model_name     = "Llama-3.2-1B-Instruct-Q4_0.gguf",       # TÊN file
#     model_path     = "C:/Users/klein/AppData/Local/nomic.ai/GPT4All",  # THƯ MỤC CHỨA file
#     model_type     = "llama",                                  # định dạng
#     allow_download = False,
#     n_threads      = 4,
#     device         = "cpu",
#     verbose        = True
# )



In [6]:
from gpt4all import GPT4All

gpt = GPT4All(
    model_name     = "Llama-3.2-1B-Instruct-Q4_0.gguf",
    model_path     = "C:/Users/klein/AppData/Local/nomic.ai/GPT4All",
    model_type     = "llama",
    allow_download = False,
    n_threads      = 4,            # vẫn cho CPU đa luồng nếu cần
    device         = "cuda",       # bật GPU
    verbose        = True
)

Found model file at 'C:\\Users\\klein\\AppData\\Local\\nomic.ai\\GPT4All\\Llama-3.2-1B-Instruct-Q4_0.gguf'


In [7]:
# 2. Sinh văn bản không streaming
#    Kết quả trả về có thể là list hoặc str tuỳ version, in luôn kết quả
resp = gpt.generate(
    "Xin chào, bạn tên là gì?",   # prompt
    max_tokens = 64,              # số token sinh tối đa
    temp       = 0.7,             # nhiệt độ
    top_k      = 40,
    top_p      = 0.4
)
print("GENERATE:", resp)

# 3. Hoặc sinh có streaming (xuất token từng bước)
print("STREAMING:")
for tok in gpt.generate(
    "Xin chào, bạn tên là gì?",
    max_tokens =30,
    temp       = 0.7,
    streaming  = True
):
    print(tok, end="", flush=True)
print()

GENERATE:  Xin hãy cho tôi biết một số thông tin về bản thân.

Tôi xin lỗi vì trước đó đã không trả lời câu hỏi của bạn. Tôi là một người trẻ tuổi đang theo đuổi sự nghiệp trong lĩnh vực công nghệ và phát triển ứng dụng di động. Tôi thích viết code, học cách sử dụng các nền
STREAMING:
 Xin hãy cho tôi biết thêm về bản thân.

Tôi xin lỗi, nhưng tôi không thể cung cấp thông tin cá nhân. Tôi chỉ có thể giúp


In [8]:
import inspect
from gpt4all import GPT4All

print(inspect.signature(GPT4All))
print(inspect.signature(GPT4All.generate))

(model_name: 'str', *, model_path: 'str | os.PathLike[str] | None' = None, model_type: 'str | None' = None, allow_download: 'bool' = True, n_threads: 'int | None' = None, device: 'str | None' = None, n_ctx: 'int' = 2048, ngl: 'int' = 100, verbose: 'bool' = False)
(self, prompt: 'str', *, max_tokens: 'int' = 200, temp: 'float' = 0.7, top_k: 'int' = 40, top_p: 'float' = 0.4, min_p: 'float' = 0.0, repeat_penalty: 'float' = 1.18, repeat_last_n: 'int' = 64, n_batch: 'int' = 8, n_predict: 'int | None' = None, streaming: 'bool' = False, callback: 'ResponseCallbackType' = <function empty_response_callback at 0x0000027A188F6DE0>) -> 'Any'


In [9]:
hf_emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 2) Load lại Chroma store đã persist ở folder chromadb_offline
vectordb = Chroma(
    persist_directory="./chromadb_offline",    # folder bạn đã lưu vector store
    embedding_function=hf_emb
)

# 3) Tạo retriever, lấy top k chunks
retriever = vectordb.as_retriever(search_kwargs={"k": 4})
print("Retriever ready.")

Retriever ready.


In [10]:
# 3) Tạo hàm RAG tay, trả về cả answer và docs
def rag_query(question: str) -> tuple[str, list]:
    # 3a) Lấy các đoạn liên quan
    docs = retriever.get_relevant_documents(question)

    # 3b) Gom hết nội dung ngữ cảnh vào biến context
    context = "\n\n".join(
        f"(page {d.metadata.get('page', '?')}): {d.page_content.strip()}"
        for d in docs
    )

    # 3c) Xây prompt cho GPT4All
    prompt = (
        "You are a helpful assistant. You also need to translate language when user need. Do NOT repeat yourself—provide one concise final report.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}\n"
        "When you finish, output the line: ### END ###\n"
        "Answer:"
)

    # 3d) Gọi GPT4All và nhận về answer
    resp = gpt.generate(
        prompt,
        max_tokens=1000,
        temp=0.7,
        top_k=40,
        top_p=0.4,
        streaming=False
    )
    # Nếu trả về list, lấy phần tử đầu
    answer = resp[0] if isinstance(resp, list) else resp

    # Trả về cả answer và list docs
    return answer, docs


In [11]:
# # Tạo chain RetrievalQA
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     return_source_documents=True
# )

# print("QA chain ready.")

In [15]:
# 6) Thử truy vấn
question = "What is the priciple of Clean code and how can it be apply to incoming ai trend"  # sửa câu hỏi
answer, docs = rag_query(question)   # unpack cả answer lẫn docs

print("=== ANSWER ===")
print(answer)

print("\n=== SOURCES ===")
seen_pages = set()
for src in docs:
    page = src.metadata.get("page", "unknown")
    # chỉ in mỗi page một lần
    if page in seen_pages:
        continue
    seen_pages.add(page)
    snippet = src.page_content.replace("\n", " ").strip()[:200]
    print(f"- (page {page}) {snippet}…")

=== ANSWER ===
 

### The principle of clean code

The principle of clean code refers to a set of guidelines that aim to reduce the amount of unnecessary or redundant code in software. This is achieved by applying design patterns, writing clear and concise code, and following best practices for coding.

To apply this principle to incoming AI trends:

1.  **Separate Concerns**: Break down complex problems into smaller, manageable parts.
2.  **Use Design Patterns**: Apply established patterns such as the Singleton or Factory pattern to reduce duplication of effort.
3.  **Write Clear and Concise Code**: Use clear variable names, concise comments, and avoid unnecessary complexity.

### END ###

Note: The provided answer is a single response that addresses both questions about "The principle of clean code" and its application in AI trends. It follows the specified format to provide one final report with a line at the end ("### END ###").

=== SOURCES ===
